In [1]:
import tensorflow as tf
import numpy as np

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

(X, y), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_test = X_test.reshape(-1, 28*28)

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X.reshape(-1, 28*28), y, test_size=5000, random_state=42)

In [3]:
X_train.shape

(55000, 784)

In [4]:
he_init = tf.keras.initializers.he_normal()

def dnn(inputs, n_hidden_layers=5, n_neurons=100, activation=tf.nn.elu, initializer=he_init, name=None):
    with tf.variable_scope(name, "dnn"):
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs

In [5]:
n_inputs = 28 * 28 # MNIST
n_outputs = 5

reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

dnn_outputs = dnn(X)

logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
learning_rate = 0.01

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss, name="training_op")

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [7]:
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_val1 = X_val[y_val < 5]
y_val1 = y_val[y_val < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [8]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train1))
        for rnd_indices in np.array_split(rnd_idx, len(X_train1) // batch_size):
            X_batch, y_batch = X_train1[rnd_indices], y_train1[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_val1, y: y_val1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

0	Validation loss: 0.141136	Best loss: 0.141136	Accuracy: 95.75%
1	Validation loss: 0.117285	Best loss: 0.117285	Accuracy: 96.78%
2	Validation loss: 1.641644	Best loss: 0.117285	Accuracy: 21.26%
3	Validation loss: 1.665131	Best loss: 0.117285	Accuracy: 20.38%
4	Validation loss: 1.677550	Best loss: 0.117285	Accuracy: 21.26%
5	Validation loss: 1.654645	Best loss: 0.117285	Accuracy: 20.38%
6	Validation loss: 1.677153	Best loss: 0.117285	Accuracy: 21.26%
7	Validation loss: 1.629229	Best loss: 0.117285	Accuracy: 19.19%
8	Validation loss: 1.660015	Best loss: 0.117285	Accuracy: 21.26%
9	Validation loss: 1.676023	Best loss: 0.117285	Accuracy: 21.26%
10	Validation loss: 1.657234	Best loss: 0.117285	Accuracy: 20.38%
11	Validation loss: 1.653458	Best loss: 0.117285	Accuracy: 20.38%
12	Validation loss: 1.676440	Best loss: 0.117285	Accuracy: 20.38%
13	Validation loss: 1.635217	Best loss: 0.117285	Accuracy: 19.19%
14	Validation loss: 1.640324	Best loss: 0.117285	Accuracy: 18.95%
15	Validation loss: 

In [9]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.exceptions import NotFittedError

class DNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_layers=5, n_neurons=100, optimizer_class=tf.train.AdamOptimizer,
                 learning_rate=0.01, batch_size=20, activation=tf.nn.elu, initializer=None,
                 batch_norm_momentum=None, dropout_rate=None, random_state=None):
        """Initialize the DNNClassifier by simply storing all the hyperparameters."""
        self.n_hidden_layers = n_hidden_layers
        self.n_neurons = n_neurons
        self.optimizer_class = optimizer_class
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.activation = activation
        if initializer:
            self.initializer = initializer
        else:
            self.initializer = tf.keras.initializers.he_normal()
        self.batch_norm_momentum = batch_norm_momentum
        self.dropout_rate = dropout_rate
        self.random_state = random_state
        self._session = None
        
    def _dnn(self, inputs):
        """Build the hidden layers, with support for batch normalization and dropout."""
        for layer in range(self.n_hidden_layers):
            if self.dropout_rate:
                inputs = tf.layers.dropout(inputs, self.dropout_rate, training=self._training)
            inputs = tf.layers.dense(inputs, self.n_neurons,
                                     kernel_initializer=self.initializer,
                                     name="hidden%d" % (layer + 1))
            if self.batch_norm_momentum:
                inputs = tf.layers.batch_normalization(inputs, momentum=self.batch_norm_momentum,
                                                       training=self._training)
            inputs = self.activation(inputs, name="hidden%d_out" % (layer + 1))
        return inputs
    
    def _build_graph(self, n_inputs, n_outputs):
        """Build the same model as earlier"""
        if self.random_state is not None:
            tf.set_random_seed(self.random_state)
            np.random.seed(self.random_state)

        X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
        y = tf.placeholder(tf.int32, shape=(None), name="y")

        if self.batch_norm_momentum or self.dropout_rate:
            self._training = tf.placeholder_with_default(False, shape=(), name='training')
        else:
            self._training = None

        dnn_outputs = self._dnn(X)

        logits = tf.layers.dense(dnn_outputs, n_outputs, kernel_initializer=self.initializer, name="logits")
        Y_proba = tf.nn.softmax(logits, name="Y_proba")

        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                                  logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

        optimizer = self.optimizer_class(learning_rate=self.learning_rate)
        training_op = optimizer.minimize(loss)

        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        # Make the important operations available easily through instance variables
        self._X, self._y = X, y
        self._Y_proba, self._loss = Y_proba, loss
        self._training_op, self._accuracy = training_op, accuracy
        self._init, self._saver = init, saver
        
    def close_session(self):
        if self._session:
            self._session.close()
            
    def _get_model_params(self):
        """Get all variable values (used for early stopping, faster than saving to disk)"""
        with self._graph.as_default():
            gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
        return {gvar.op.name: value for gvar, value in zip(gvars, self._session.run(gvars))}

    def _restore_model_params(self, model_params):
        """Set all variables to the given values (for early stopping, faster than loading from disk)"""
        gvar_names = list(model_params.keys())
        assign_ops = {gvar_name: self._graph.get_operation_by_name(gvar_name + "/Assign")
                      for gvar_name in gvar_names}
        init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
        feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
        self._session.run(assign_ops, feed_dict=feed_dict)

    def fit(self, X, y, n_epochs=100, X_valid=None, y_valid=None):
        """Fit the model to the training set. If X_valid and y_valid are provided, use early stopping."""
        self.close_session()

        # infer n_inputs and n_outputs from the training set.
        n_inputs = X.shape[1]
        self.classes_ = np.unique(y)
        n_outputs = len(self.classes_)
        
        # Translate the labels vector to a vector of sorted class indices, containing
        # integers from 0 to n_outputs - 1.
        # For example, if y is equal to [8, 8, 9, 5, 7, 6, 6, 6], then the sorted class
        # labels (self.classes_) will be equal to [5, 6, 7, 8, 9], and the labels vector
        # will be translated to [3, 3, 4, 0, 2, 1, 1, 1]
        self.class_to_index_ = {label: index
                                for index, label in enumerate(self.classes_)}
        y = np.array([self.class_to_index_[label]
                      for label in y], dtype=np.int32)
        
        self._graph = tf.Graph()
        with self._graph.as_default():
            self._build_graph(n_inputs, n_outputs)
            # extra ops for batch normalization
            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        # needed in case of early stopping
        max_checks_without_progress = 20
        checks_without_progress = 0
        best_loss = np.infty
        best_params = None
        
        # Now train the model!
        self._session = tf.Session(graph=self._graph)
        with self._session.as_default() as sess:
            self._init.run()
            for epoch in range(n_epochs):
                rnd_idx = np.random.permutation(len(X))
                for rnd_indices in np.array_split(rnd_idx, len(X) // self.batch_size):
                    X_batch, y_batch = X[rnd_indices], y[rnd_indices]
                    feed_dict = {self._X: X_batch, self._y: y_batch}
                    if self._training is not None:
                        feed_dict[self._training] = True
                    sess.run(self._training_op, feed_dict=feed_dict)
                    if extra_update_ops:
                        sess.run(extra_update_ops, feed_dict=feed_dict)
                if X_valid is not None and y_valid is not None:
                    loss_val, acc_val = sess.run([self._loss, self._accuracy],
                                                 feed_dict={self._X: X_valid,
                                                            self._y: y_valid})
                    if loss_val < best_loss:
                        best_params = self._get_model_params()
                        best_loss = loss_val
                        checks_without_progress = 0
                    else:
                        checks_without_progress += 1
                    print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_val, best_loss, acc_val * 100))
                    if checks_without_progress > max_checks_without_progress:
                        print("Early stopping!")
                        break
                else:
                    loss_train, acc_train = sess.run([self._loss, self._accuracy],
                                                     feed_dict={self._X: X_batch,
                                                                self._y: y_batch})
                    print("{}\tLast training batch loss: {:.6f}\tAccuracy: {:.2f}%".format(
                        epoch, loss_train, acc_train * 100))
            # If we used early stopping then rollback to the best model found
            if best_params:
                self._restore_model_params(best_params)
            return self

    def predict_proba(self, X):
        if not self._session:
            raise NotFittedError("This %s instance is not fitted yet" % self.__class__.__name__)
        with self._session.as_default() as sess:
            return self._Y_proba.eval(feed_dict={self._X: X})

    def predict(self, X):
        class_indices = np.argmax(self.predict_proba(X), axis=1)
        return np.array([[self.classes_[class_index]]
                         for class_index in class_indices], np.int32)

    def save(self, path):
        self._saver.save(self._session, path)

In [10]:
dnn_clf = DNNClassifier(random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_val1, y_valid=y_val1)

0	Validation loss: 0.141136	Best loss: 0.141136	Accuracy: 95.75%
1	Validation loss: 0.117285	Best loss: 0.117285	Accuracy: 96.78%
2	Validation loss: 1.641644	Best loss: 0.117285	Accuracy: 21.26%
3	Validation loss: 1.665131	Best loss: 0.117285	Accuracy: 20.38%
4	Validation loss: 1.677550	Best loss: 0.117285	Accuracy: 21.26%
5	Validation loss: 1.654645	Best loss: 0.117285	Accuracy: 20.38%
6	Validation loss: 1.677153	Best loss: 0.117285	Accuracy: 21.26%
7	Validation loss: 1.629229	Best loss: 0.117285	Accuracy: 19.19%
8	Validation loss: 1.660015	Best loss: 0.117285	Accuracy: 21.26%
9	Validation loss: 1.676023	Best loss: 0.117285	Accuracy: 21.26%
10	Validation loss: 1.657234	Best loss: 0.117285	Accuracy: 20.38%
11	Validation loss: 1.653458	Best loss: 0.117285	Accuracy: 20.38%
12	Validation loss: 1.676440	Best loss: 0.117285	Accuracy: 20.38%
13	Validation loss: 1.635217	Best loss: 0.117285	Accuracy: 19.19%
14	Validation loss: 1.640324	Best loss: 0.117285	Accuracy: 18.95%
15	Validation loss: 

DNNClassifier(activation=<function elu at 0x00000141B28E7730>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000141B674A128>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [11]:
from sklearn.metrics import accuracy_score

y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9676979957190115

In [12]:
from sklearn.model_selection import RandomizedSearchCV
from functools import partial

def leaky_relu(alpha=0.01):
    def parametrized_leaky_relu(z, name=None):
        return tf.maximum(alpha * z, z, name=name)
    return parametrized_leaky_relu

param_distribs = {
    "n_neurons": [10, 30, 50, 70, 90, 100, 120, 140, 160],
    "batch_size": [10, 50, 100, 500],
    "learning_rate": [0.01, 0.02, 0.05, 0.1],
    "activation": [tf.nn.relu, tf.nn.elu, leaky_relu(alpha=0.01), leaky_relu(alpha=0.1)],
    "n_hidden_layers": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "optimizer_class": [tf.train.AdamOptimizer, partial(tf.train.MomentumOptimizer, momentum=0.95)],
}

rnd_search = RandomizedSearchCV(DNNClassifier(random_state=42), param_distribs, n_iter=50,
                                cv=3, random_state=42, verbose=2)
rnd_search.fit(X_train1, y_train1, X_valid=X_val1, y_valid=y_val1, n_epochs=1000)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>, n_neurons=160, n_hidden_layers=7, learning_rate=0.01, batch_size=50, activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000141B82B2C80> 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0	Validation loss: 0.122777	Best loss: 0.122777	Accuracy: 96.82%
1	Validation loss: 0.141832	Best loss: 0.122777	Accuracy: 96.15%
2	Validation loss: 0.097214	Best loss: 0.097214	Accuracy: 97.81%
3	Validation loss: 0.099893	Best loss: 0.097214	Accuracy: 97.50%
4	Validation loss: 0.086266	Best loss: 0.086266	Accuracy: 98.13%
5	Validation loss: 0.082039	Best loss: 0.082039	Accuracy: 97.97%
6	Validation loss: 10114279.000000	Best loss: 0.082039	Accuracy: 20.26%
7	Validation loss: 7924.826172	Best loss: 0.082039	Accuracy: 50.70%
8	Validation loss: 4953.498047	Best loss: 0.082039	Accuracy: 65.63%
9	Validation loss: 1400.741211	Best loss: 0.082039	Accuracy: 74.45%
10	Validation loss: 478.644012	Best loss: 0.082039	Accuracy: 80.10%
11	Validation loss: 734.261780	Best loss: 0.082039	Accuracy: 76.12%
12	Validation loss: 1280.696411	Best loss: 0.082039	Accuracy: 73.90%
13	Validation loss: 323.885773	Best loss: 0.082039	Accuracy: 84.78%
14	Validation loss: 535.937927	Best loss: 0.082039	Accuracy: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   40.1s remaining:    0.0s


0	Validation loss: 0.122991	Best loss: 0.122991	Accuracy: 96.82%
1	Validation loss: 0.122681	Best loss: 0.122681	Accuracy: 97.02%
2	Validation loss: 0.172006	Best loss: 0.122681	Accuracy: 94.99%
3	Validation loss: 0.088526	Best loss: 0.088526	Accuracy: 97.42%
4	Validation loss: 0.093071	Best loss: 0.088526	Accuracy: 97.85%
5	Validation loss: 0.078861	Best loss: 0.078861	Accuracy: 98.13%
6	Validation loss: 0.129873	Best loss: 0.078861	Accuracy: 96.34%
7	Validation loss: 165294.953125	Best loss: 0.078861	Accuracy: 19.47%
8	Validation loss: 2080.961426	Best loss: 0.078861	Accuracy: 45.01%
9	Validation loss: 781.630188	Best loss: 0.078861	Accuracy: 51.13%
10	Validation loss: 262.895996	Best loss: 0.078861	Accuracy: 62.42%
11	Validation loss: 260.195251	Best loss: 0.078861	Accuracy: 54.23%
12	Validation loss: 153.949066	Best loss: 0.078861	Accuracy: 70.52%
13	Validation loss: 86.218781	Best loss: 0.078861	Accuracy: 80.69%
14	Validation loss: 192.698456	Best loss: 0.078861	Accuracy: 63.57%
1

...

24	Validation loss: 0.170537	Best loss: 0.170537	Accuracy: 95.07%
25	Validation loss: 0.171164	Best loss: 0.170537	Accuracy: 95.15%
26	Validation loss: 0.167266	Best loss: 0.167266	Accuracy: 95.15%
27	Validation loss: 0.170541	Best loss: 0.167266	Accuracy: 94.87%
28	Validation loss: 0.176498	Best loss: 0.167266	Accuracy: 95.27%
29	Validation loss: 0.158339	Best loss: 0.158339	Accuracy: 95.39%
30	Validation loss: 0.169654	Best loss: 0.158339	Accuracy: 95.11%
31	Validation loss: 0.230167	Best loss: 0.158339	Accuracy: 92.17%
32	Validation loss: 0.171946	Best loss: 0.158339	Accuracy: 94.28%
33	Validation loss: 0.169765	Best loss: 0.158339	Accuracy: 94.68%
34	Validation loss: 0.174533	Best loss: 0.158339	Accuracy: 94.60%
35	Validation loss: 0.163645	Best loss: 0.158339	Accuracy: 94.99%
36	Validation loss: 0.161877	Best loss: 0.158339	Accuracy: 95.35%
37	Validation loss: 0.171375	Best loss: 0.158339	Accuracy: 94.84%
38	Validation loss: 0.154020	Best loss: 0.154020	Accuracy: 95.27%
39	Validat

3	Validation loss: 1672750301184.000000	Best loss: 951973117952.000000	Accuracy: 18.91%
4	Validation loss: 169282863104.000000	Best loss: 169282863104.000000	Accuracy: 36.15%
5	Validation loss: 750233518080.000000	Best loss: 169282863104.000000	Accuracy: 20.46%
6	Validation loss: 24363042816.000000	Best loss: 24363042816.000000	Accuracy: 53.99%
7	Validation loss: 51029680128.000000	Best loss: 24363042816.000000	Accuracy: 46.92%
8	Validation loss: 78369112064.000000	Best loss: 24363042816.000000	Accuracy: 38.50%
9	Validation loss: 47056187392.000000	Best loss: 24363042816.000000	Accuracy: 45.93%
10	Validation loss: 61492260864.000000	Best loss: 24363042816.000000	Accuracy: 43.19%
11	Validation loss: 68011220992.000000	Best loss: 24363042816.000000	Accuracy: 46.40%
12	Validation loss: nan	Best loss: 24363042816.000000	Accuracy: 0.00%
13	Validation loss: nan	Best loss: 24363042816.000000	Accuracy: 0.00%
14	Validation loss: nan	Best loss: 24363042816.000000	Accuracy: 0.00%
15	Validation lo

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 81.3min finished


0	Validation loss: 0.107391	Best loss: 0.107391	Accuracy: 96.78%
1	Validation loss: 0.090649	Best loss: 0.090649	Accuracy: 97.10%
2	Validation loss: 0.087193	Best loss: 0.087193	Accuracy: 97.42%
3	Validation loss: 0.082617	Best loss: 0.082617	Accuracy: 97.66%
4	Validation loss: 0.051717	Best loss: 0.051717	Accuracy: 98.41%
5	Validation loss: 0.045854	Best loss: 0.045854	Accuracy: 98.57%
6	Validation loss: 0.058600	Best loss: 0.045854	Accuracy: 98.05%
7	Validation loss: 0.059244	Best loss: 0.045854	Accuracy: 98.17%
8	Validation loss: 0.051322	Best loss: 0.045854	Accuracy: 98.57%
9	Validation loss: 0.059947	Best loss: 0.045854	Accuracy: 98.21%
10	Validation loss: 0.055041	Best loss: 0.045854	Accuracy: 98.53%
11	Validation loss: 0.053472	Best loss: 0.045854	Accuracy: 98.37%
12	Validation loss: 0.040740	Best loss: 0.040740	Accuracy: 98.89%
13	Validation loss: 0.041960	Best loss: 0.040740	Accuracy: 98.93%
14	Validation loss: 0.052720	Best loss: 0.040740	Accuracy: 98.81%
15	Validation loss: 

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=DNNClassifier(activation=<function elu at 0x00000141B28E7730>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000141B8627BA8>,
       learning_rate=0.01, n_hidden_layers=5, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42),
          fit_params=None, iid='warn', n_iter=50, n_jobs=None,
          param_distributions={'n_neurons': [10, 30, 50, 70, 90, 100, 120, 140, 160], 'batch_size': [10, 50, 100, 500], 'learning_rate': [0.01, 0.02, 0.05, 0.1], 'activation': [<function relu at 0x00000141B28F8C80>, <function elu at 0x00000141B28E7730>, <function leaky_relu.<locals>.parametrized_leaky_relu at... functools.partial(<class 'tensorflow.python.training.momentum.MomentumOptimizer'>, momentum=0.95)]},
          pre_dispatch='2*n_jobs', random_state=42

In [14]:
rnd_search.best_params_

{'optimizer_class': tensorflow.python.training.adam.AdamOptimizer,
 'n_neurons': 160,
 'n_hidden_layers': 10,
 'learning_rate': 0.01,
 'batch_size': 100,
 'activation': <function __main__.leaky_relu.<locals>.parametrized_leaky_relu(z, name=None)>}

In [15]:
y_pred = rnd_search.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9904650710254913

In [16]:
rnd_search.best_estimator_.save("./my_best_mnist_model_0_to_4")

In [21]:
dnn_clf = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=100, learning_rate=0.01, n_hidden_layers=10,
                        n_neurons=160, random_state=42)
dnn_clf.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_val1, y_valid=y_val1)

0	Validation loss: 0.107391	Best loss: 0.107391	Accuracy: 96.78%
1	Validation loss: 0.090649	Best loss: 0.090649	Accuracy: 97.10%
2	Validation loss: 0.087193	Best loss: 0.087193	Accuracy: 97.42%
3	Validation loss: 0.082617	Best loss: 0.082617	Accuracy: 97.66%
4	Validation loss: 0.051717	Best loss: 0.051717	Accuracy: 98.41%
5	Validation loss: 0.045854	Best loss: 0.045854	Accuracy: 98.57%
6	Validation loss: 0.058600	Best loss: 0.045854	Accuracy: 98.05%
7	Validation loss: 0.059244	Best loss: 0.045854	Accuracy: 98.17%
8	Validation loss: 0.051322	Best loss: 0.045854	Accuracy: 98.57%
9	Validation loss: 0.059947	Best loss: 0.045854	Accuracy: 98.21%
10	Validation loss: 0.055041	Best loss: 0.045854	Accuracy: 98.53%
11	Validation loss: 0.053472	Best loss: 0.045854	Accuracy: 98.37%
12	Validation loss: 0.040740	Best loss: 0.040740	Accuracy: 98.89%
13	Validation loss: 0.041960	Best loss: 0.040740	Accuracy: 98.93%
14	Validation loss: 0.052720	Best loss: 0.040740	Accuracy: 98.81%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000141C259FD08>,
       batch_norm_momentum=None, batch_size=100, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000141B9034DA0>,
       learning_rate=0.01, n_hidden_layers=10, n_neurons=160,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [22]:
y_pred = dnn_clf.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9904650710254913

In [23]:
dnn_clf_bn = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=100, learning_rate=0.01,
                           n_hidden_layers=10, n_neurons=160, random_state=42,
                           batch_norm_momentum=0.95)
dnn_clf_bn.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_val1, y_valid=y_val1)

Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.
0	Validation loss: 0.080087	Best loss: 0.080087	Accuracy: 98.37%
1	Validation loss: 0.052201	Best loss: 0.052201	Accuracy: 98.41%
2	Validation loss: 0.037549	Best loss: 0.037549	Accuracy: 98.97%
3	Validation loss: 0.034184	Best loss: 0.034184	Accuracy: 98.69%
4	Validation loss: 0.025228	Best loss: 0.025228	Accuracy: 99.32%
5	Validation loss: 0.021188	Best loss: 0.021188	Accuracy: 99.52%
6	Validation loss: 0.048494	Best loss: 0.021188	Accuracy: 98.97%
7	Validation loss: 0.042446	Best loss: 0.021188	Accuracy: 98.97%
8	Validation loss: 0.058218	Best loss: 0.021188	Accuracy: 98.25%
9	Validation loss: 0.023091	Best loss: 0.021188	Accuracy: 99.25%
10	Validation loss: 0.018064	Best loss: 0.018064	Accuracy: 99.48%
11	Validation loss: 0.050662	Best loss: 0.018064	Accuracy: 98.77%
12	Validation loss: 0.038617	Best loss: 0.018064	Accuracy: 98.97%
13	Validation loss: 0.023071	Be

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000141B8E39378>,
       batch_norm_momentum=0.95, batch_size=100, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000141B8907358>,
       learning_rate=0.01, n_hidden_layers=10, n_neurons=160,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [24]:
y_pred = dnn_clf_bn.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9935785172212492

In [27]:
y_pred = dnn_clf_bn.predict(X_train1)
accuracy_score(y_train1, y_pred)

0.9987891306670466

In [31]:
dnn_clf_dropout = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=100, learning_rate=0.01,
                                n_hidden_layers=10, n_neurons=160, random_state=42,
                                dropout_rate=0.2)
dnn_clf_dropout.fit(X_train1, y_train1, n_epochs=1000, X_valid=X_val1, y_valid=y_val1)

0	Validation loss: 0.543348	Best loss: 0.543348	Accuracy: 76.68%
1	Validation loss: 0.179749	Best loss: 0.179749	Accuracy: 93.96%
2	Validation loss: 0.121876	Best loss: 0.121876	Accuracy: 95.95%
3	Validation loss: 0.101872	Best loss: 0.101872	Accuracy: 96.94%
4	Validation loss: 0.110581	Best loss: 0.101872	Accuracy: 96.94%
5	Validation loss: 0.110005	Best loss: 0.101872	Accuracy: 96.34%
6	Validation loss: 0.080682	Best loss: 0.080682	Accuracy: 97.54%
7	Validation loss: 0.071987	Best loss: 0.071987	Accuracy: 98.29%
8	Validation loss: 0.065854	Best loss: 0.065854	Accuracy: 98.25%
9	Validation loss: 0.098469	Best loss: 0.065854	Accuracy: 97.26%
10	Validation loss: 0.063641	Best loss: 0.063641	Accuracy: 98.17%
11	Validation loss: 593800.562500	Best loss: 0.063641	Accuracy: 20.22%
12	Validation loss: 2363.415283	Best loss: 0.063641	Accuracy: 19.35%
13	Validation loss: 5073.772949	Best loss: 0.063641	Accuracy: 19.19%
14	Validation loss: 2075.687500	Best loss: 0.063641	Accuracy: 20.22%
15	Val

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000141CD64A8C8>,
       batch_norm_momentum=None, batch_size=100, dropout_rate=0.2,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000141CC578908>,
       learning_rate=0.01, n_hidden_layers=10, n_neurons=160,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [32]:
y_pred = dnn_clf_dropout.predict(X_test1)
accuracy_score(y_test1, y_pred)

0.9838489978595057

In [33]:
y_pred = dnn_clf_dropout.predict(X_train1)
accuracy_score(y_train1, y_pred)

0.9835464225933972

In [34]:
dnn_clf_bn.save("./my_best_mnist_model_0_to_4")

In [35]:
reset_graph()

restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")
loss = tf.get_default_graph().get_tensor_by_name("loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("Y_proba:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")

In [36]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

In [37]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

In [39]:
X_train2_full = X_train[y_train >= 5]
y_train2_full = y_train[y_train >= 5] - 5
X_val2_full = X_val[y_val >= 5]
y_val2_full = y_val[y_val >= 5] - 5
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >= 5] - 5

In [40]:
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

In [41]:
X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_val2, y_val2 = sample_n_instances_per_class(X_val2_full, y_val2_full, n=30)

In [42]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
    t0 = time.time()
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_val2, y: y_val2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.803708	Best loss: 1.803708	Accuracy: 34.00%
1	Validation loss: 1.343907	Best loss: 1.343907	Accuracy: 50.67%
2	Validation loss: 1.247623	Best loss: 1.247623	Accuracy: 56.00%
3	Validation loss: 1.209732	Best loss: 1.209732	Accuracy: 56.67%
4	Validation loss: 1.190325	Best loss: 1.190325	Accuracy: 56.67%
5	Validation loss: 1.188166	Best loss: 1.188166	Accuracy: 56.00%
6	Validation loss: 1.179093	Best loss: 1.179093	Accuracy: 58.67%
7	Validation loss: 1.171937	Best loss: 1.171937	Accuracy: 58.00%
8	Validation loss: 1.170886	Best loss: 1.170886	Accuracy: 58.67%
9	Validation loss: 1.169981	Best loss: 1.169981	Accuracy: 56.67%
10	Validation loss: 1.181657	Best loss: 1.169981	Accuracy: 57.33%
11	Validation loss: 1.160368	Best loss: 1.160368	Accuracy: 60.00%
12	Validation loss: 1.165949	Best loss: 1.160368	Accuracy: 56.00%
13	Validation loss: 1.161836	Best loss: 1.160368	Accuracy: 60.00%
14	Validation l

In [43]:
hidden5_out = tf.get_default_graph().get_tensor_by_name("hidden5_out:0")

In [44]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
    t0 = time.time()
    
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_val2, y: y_val2})
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_val2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.765948	Best loss: 1.765948	Accuracy: 36.00%
1	Validation loss: 1.350226	Best loss: 1.350226	Accuracy: 49.33%
2	Validation loss: 1.247587	Best loss: 1.247587	Accuracy: 57.33%
3	Validation loss: 1.208008	Best loss: 1.208008	Accuracy: 57.33%
4	Validation loss: 1.188360	Best loss: 1.188360	Accuracy: 56.67%
5	Validation loss: 1.176604	Best loss: 1.176604	Accuracy: 58.00%
6	Validation loss: 1.178279	Best loss: 1.176604	Accuracy: 57.33%
7	Validation loss: 1.196646	Best loss: 1.176604	Accuracy: 55.33%
8	Validation loss: 1.171340	Best loss: 1.171340	Accuracy: 58.67%
9	Validation loss: 1.173495	Best loss: 1.171340	Accuracy: 60.00%
10	Validation loss: 1.166227	Best loss: 1.166227	Accuracy: 54.67%
11	Validation loss: 1.167260	Best loss: 1.166227	Accuracy: 58.67%
12	Validation loss: 1.155831	Best loss: 1.155831	Accuracy: 58.00%
13	Validation loss: 1.152319	Best loss: 1.152319	Accuracy: 59.33%
14	Validation l

In [45]:
reset_graph()

n_outputs = 5

restore_saver = tf.train.import_meta_graph("./my_best_mnist_model_0_to_4.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden4_out = tf.get_default_graph().get_tensor_by_name("hidden4_out:0")
logits = tf.layers.dense(hidden4_out, n_outputs, kernel_initializer=he_init, name="new_logits")
Y_proba = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [46]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)

init = tf.global_variables_initializer()
four_frozen_saver = tf.train.Saver()

In [47]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_best_mnist_model_0_to_4")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_val2, y: y_val2})
        if loss_val < best_loss:
            save_path = four_frozen_saver.save(sess, "./my_mnist_model_5_to_9_four_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_best_mnist_model_0_to_4
0	Validation loss: 1.223293	Best loss: 1.223293	Accuracy: 50.67%
1	Validation loss: 1.137847	Best loss: 1.137847	Accuracy: 56.00%
2	Validation loss: 1.117902	Best loss: 1.117902	Accuracy: 60.00%
3	Validation loss: 1.112414	Best loss: 1.112414	Accuracy: 56.00%
4	Validation loss: 1.069100	Best loss: 1.069100	Accuracy: 60.67%
5	Validation loss: 1.110934	Best loss: 1.069100	Accuracy: 56.67%
6	Validation loss: 1.058996	Best loss: 1.058996	Accuracy: 61.33%
7	Validation loss: 1.054920	Best loss: 1.054920	Accuracy: 62.67%
8	Validation loss: 1.058119	Best loss: 1.054920	Accuracy: 60.00%
9	Validation loss: 1.069787	Best loss: 1.054920	Accuracy: 63.33%
10	Validation loss: 1.139812	Best loss: 1.054920	Accuracy: 62.00%
11	Validation loss: 1.137339	Best loss: 1.054920	Accuracy: 58.00%
12	Validation loss: 1.118403	Best loss: 1.054920	Accuracy: 60.67%
13	Validation loss: 1.093510	Best loss: 1.054920	Accuracy: 63.33%
14	Validation l

In [48]:
learning_rate = 0.01

unfrozen_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|new_logits")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam3")
training_op = optimizer.minimize(loss, var_list=unfrozen_vars)

init = tf.global_variables_initializer()
two_frozen_saver = tf.train.Saver()

In [49]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    four_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_four_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_val2, y: y_val2})
        if loss_val < best_loss:
            save_path = two_frozen_saver.save(sess, "./my_mnist_model_5_to_9_two_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_four_frozen
0	Validation loss: 0.923258	Best loss: 0.923258	Accuracy: 69.33%
1	Validation loss: 0.915593	Best loss: 0.915593	Accuracy: 74.00%
2	Validation loss: 0.766074	Best loss: 0.766074	Accuracy: 77.33%
3	Validation loss: 0.765655	Best loss: 0.765655	Accuracy: 78.00%
4	Validation loss: 0.756302	Best loss: 0.756302	Accuracy: 77.33%
5	Validation loss: 0.767173	Best loss: 0.756302	Accuracy: 82.00%
6	Validation loss: 0.737163	Best loss: 0.737163	Accuracy: 84.67%
7	Validation loss: 0.811674	Best loss: 0.737163	Accuracy: 83.33%
8	Validation loss: 0.743897	Best loss: 0.737163	Accuracy: 84.00%
9	Validation loss: 0.799730	Best loss: 0.737163	Accuracy: 84.67%
10	Validation loss: 0.820891	Best loss: 0.737163	Accuracy: 85.33%
11	Validation loss: 0.863008	Best loss: 0.737163	Accuracy: 86.00%
12	Validation loss: 0.874656	Best loss: 0.737163	Accuracy: 86.67%
13	Validation loss: 0.845135	Best loss: 0.737163	Accuracy: 85.33%
14	Valid

In [50]:
learning_rate = 0.01

optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam4")
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
no_frozen_saver = tf.train.Saver()

In [51]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    two_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_two_frozen")
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_val2, y: y_val2})
        if loss_val < best_loss:
            save_path = no_frozen_saver.save(sess, "./my_mnist_model_5_to_9_no_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    no_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_no_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from ./my_mnist_model_5_to_9_two_frozen
0	Validation loss: 0.791083	Best loss: 0.791083	Accuracy: 83.33%
1	Validation loss: 0.691287	Best loss: 0.691287	Accuracy: 88.67%
2	Validation loss: 0.773437	Best loss: 0.691287	Accuracy: 90.00%
3	Validation loss: 0.745614	Best loss: 0.691287	Accuracy: 90.00%
4	Validation loss: 0.566180	Best loss: 0.566180	Accuracy: 90.00%
5	Validation loss: 0.691811	Best loss: 0.566180	Accuracy: 88.00%
6	Validation loss: 0.658004	Best loss: 0.566180	Accuracy: 88.00%
7	Validation loss: 0.706013	Best loss: 0.566180	Accuracy: 89.33%
8	Validation loss: 0.797545	Best loss: 0.566180	Accuracy: 89.33%
9	Validation loss: 0.752342	Best loss: 0.566180	Accuracy: 90.00%
10	Validation loss: 0.716509	Best loss: 0.566180	Accuracy: 90.00%
11	Validation loss: 0.719630	Best loss: 0.566180	Accuracy: 90.67%
12	Validation loss: 0.722439	Best loss: 0.566180	Accuracy: 90.00%
13	Validation loss: 0.726229	Best loss: 0.566180	Accuracy: 90.00%
14	Valida

In [52]:
dnn_clf_5_to_9 = DNNClassifier(n_hidden_layers=4, random_state=42)
dnn_clf_5_to_9.fit(X_train2, y_train2, n_epochs=1000, X_valid=X_val2, y_valid=y_val2)

0	Validation loss: 22.188021	Best loss: 22.188021	Accuracy: 62.67%
1	Validation loss: 9.014125	Best loss: 9.014125	Accuracy: 74.00%
2	Validation loss: 5.014882	Best loss: 5.014882	Accuracy: 80.67%
3	Validation loss: 2.867158	Best loss: 2.867158	Accuracy: 84.67%
4	Validation loss: 1.936761	Best loss: 1.936761	Accuracy: 89.33%
5	Validation loss: 1.846609	Best loss: 1.846609	Accuracy: 86.00%
6	Validation loss: 1.934107	Best loss: 1.846609	Accuracy: 86.67%
7	Validation loss: 1.690617	Best loss: 1.690617	Accuracy: 89.33%
8	Validation loss: 1.593072	Best loss: 1.593072	Accuracy: 89.33%
9	Validation loss: 1.481518	Best loss: 1.481518	Accuracy: 87.33%
10	Validation loss: 1.547439	Best loss: 1.481518	Accuracy: 89.33%
11	Validation loss: 1.421610	Best loss: 1.421610	Accuracy: 90.67%
12	Validation loss: 1.413781	Best loss: 1.413781	Accuracy: 90.67%
13	Validation loss: 1.436234	Best loss: 1.413781	Accuracy: 91.33%
14	Validation loss: 1.440779	Best loss: 1.413781	Accuracy: 91.33%
15	Validation loss

DNNClassifier(activation=<function elu at 0x00000141B28E7730>,
       batch_norm_momentum=None, batch_size=20, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000141DA6C9630>,
       learning_rate=0.01, n_hidden_layers=4, n_neurons=100,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [53]:
y_pred = dnn_clf_5_to_9.predict(X_test2)
accuracy_score(y_test2, y_pred)

0.9002262908866488

In [56]:
dnn_clf = DNNClassifier(activation=leaky_relu(alpha=0.1), batch_size=100, learning_rate=0.01, n_hidden_layers=10,
                        n_neurons=160, random_state=42,
                           batch_norm_momentum=0.95)
dnn_clf.fit(X_train, y_train, n_epochs=1000, X_valid=X_val, y_valid=y_val)

0	Validation loss: 0.174442	Best loss: 0.174442	Accuracy: 95.28%
1	Validation loss: 0.126714	Best loss: 0.126714	Accuracy: 96.28%
2	Validation loss: 0.101504	Best loss: 0.101504	Accuracy: 97.00%
3	Validation loss: 0.094064	Best loss: 0.094064	Accuracy: 97.20%
4	Validation loss: 0.114725	Best loss: 0.094064	Accuracy: 96.82%
5	Validation loss: 0.173419	Best loss: 0.094064	Accuracy: 95.98%
6	Validation loss: 0.078967	Best loss: 0.078967	Accuracy: 97.72%
7	Validation loss: 0.086595	Best loss: 0.078967	Accuracy: 97.68%
8	Validation loss: 0.095683	Best loss: 0.078967	Accuracy: 97.32%
9	Validation loss: 0.091878	Best loss: 0.078967	Accuracy: 97.56%
10	Validation loss: 0.085370	Best loss: 0.078967	Accuracy: 97.72%
11	Validation loss: 0.073202	Best loss: 0.073202	Accuracy: 98.16%
12	Validation loss: 0.079808	Best loss: 0.073202	Accuracy: 97.64%
13	Validation loss: 0.067097	Best loss: 0.067097	Accuracy: 98.06%
14	Validation loss: 0.072890	Best loss: 0.067097	Accuracy: 97.98%
15	Validation loss: 

DNNClassifier(activation=<function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000141DA738E18>,
       batch_norm_momentum=0.95, batch_size=100, dropout_rate=None,
       initializer=<tensorflow.python.ops.init_ops.VarianceScaling object at 0x00000141DC1E00B8>,
       learning_rate=0.01, n_hidden_layers=10, n_neurons=160,
       optimizer_class=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       random_state=42)

In [62]:
y_pred = dnn_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9813